In [1]:
# importing libraries
import torch
from torchvision import datasets,transforms
from face_recognition import FaceFeaturesExtractor,preprocessing, FaceRecogniser
from PIL import Image
import numpy as np
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import os
from PIL import ImageDraw, ImageFont

In [2]:
!ls

embeddings	  face_recognition.ipynb  images   README.md  Train_model
face_recognition  fonts			  LICENSE  results


In [3]:
# read files in images and generating embeddings.
torch.set_grad_enabled(False)

In [4]:
image_path = 'images/'
path_embeddings = 'embeddings/'

In [5]:
features_extractor = FaceFeaturesExtractor()
dataset = datasets.ImageFolder(image_path)


In [6]:
def generate_new_embeddings(image_path,path_embeddings):
    transform = transforms.Compose([
        preprocessing.ExifOrientationNormalize(),
        transforms.Resize(1024)
    ])
    embeddings = []
    labels = []
    for img_path, label in dataset.samples:
        print(img_path)
        _, embedding = features_extractor(transform(Image.open(img_path).convert('RGB')))
        if embedding is None:
            print("Could not find face on {}".format(img_path))
            continue
        if embedding.shape[0] > 1:
            print("Multiple faces detected for {}, taking one with highest probability".format(img_path))
            embedding = embedding[0, :]
        embeddings.append(embedding.flatten())
        labels.append(label)
    Embeddings = np.stack(embeddings)
    idx_to_class = {v: k for k, v in dataset.class_to_idx.items()}
    np.savetxt(path_embeddings+'embeddings.txt', embeddings)
    np.savetxt(path_embeddings+ 'labels.txt', np.array(labels, dtype=np.str).reshape(-1, 1), fmt="%s")
    joblib.dump(idx_to_class, path_embeddings + 'idx_to_class.pkl')

In [7]:
generate_new_embeddings(image_path,path_embeddings)

images/Jeans/WIN_20211005_17_45_23_Pro (2).jpg
images/Jeans/WIN_20211005_17_45_26_Pro.jpg
images/Jeans/WIN_20211005_17_45_28_Pro.jpg
images/Jeans/WIN_20211005_17_45_30_Pro.jpg
images/Jeans/WIN_20211005_17_45_36_Pro.jpg
images/Jeans/WIN_20211005_17_45_37_Pro.jpg
images/Jeans/WIN_20211005_17_45_39_Pro.jpg
images/Juan_diego/WIN_20211005_17_38_05_Pro.jpg
images/Juan_diego/WIN_20211005_17_38_11_Pro.jpg
images/Juan_diego/WIN_20211005_17_38_13_Pro.jpg
images/Juan_diego/WIN_20211005_17_38_16_Pro.jpg
images/Juan_diego/WIN_20211005_17_38_22_Pro.jpg
images/Juan_diego/WIN_20211005_17_38_24_Pro.jpg
images/Juan_diego/WIN_20211005_17_38_25_Pro.jpg
images/daniel/WIN_20211005_06_53_27_Pro.jpg
images/daniel/WIN_20211005_06_53_29_Pro.jpg
images/daniel/WIN_20211005_06_53_31_Pro.jpg
images/daniel/daniel_1.jpg
images/daniel/daniel_2.jpg
images/daniel/daniel_3.jpg
images/daniel/daniel_4.jpg
images/daniel/daniel_5.jpg
images/daniel/daniel_6.jpg
images/daniel/daniel_7.jpg
images/daniel_cepeda/WIN_20211005_18_0

In [8]:
#load saved data
embeddings_path = 'embeddings/embeddings.txt'
labels_path = 'embeddings/labels.txt'
idx_to_class_path = 'embeddings/idx_to_class.pkl'
embeddings = np.loadtxt(embeddings_path)
labels = np.loadtxt(labels_path, dtype='int').tolist()
idx_to_class = joblib.load(idx_to_class_path)

In [9]:
#train a model
model= LogisticRegression(solver='lbfgs', multi_class='multinomial', C=10, max_iter=10000)
clf = GridSearchCV(
    estimator=model,
    param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
    cv=3
)

In [10]:
clf.fit(embeddings, labels)
best_model = clf.best_estimator_

In [11]:
model_path= 'Train_model.pkl'
joblib.dump(FaceRecogniser(features_extractor, best_model, idx_to_class), model_path)

['Train_model.pkl']

In [12]:
print(clf.cv_results_['mean_test_score'])
print(clf.cv_results_['std_test_score'])

[0.25091575 0.25091575 0.27655678 0.9029304  0.97619048 0.97619048
 0.97619048]
[0.04070716 0.04070716 0.07670473 0.08863409 0.03367175 0.03367175
 0.03367175]


In [14]:
def draw_bb_on_img(faces, img):
    draw = ImageDraw.Draw(img)
    fs = max(20, round(img.size[0] * img.size[1] * 0.000005))
    font = ImageFont.truetype('fonts/font.ttf', fs)
    margin = 5

    for face in faces:
        print(face.top_prediction.confidence)
        if face.top_prediction.confidence > 0.7:
            text = "%s %.2f%%" % (face.top_prediction.label.upper(), face.top_prediction.confidence * 100)
        else:
            text='Desconocido'
        text_size = font.getsize(text)

        # bounding box
        draw.rectangle(
            (
                (int(face.bb.left), int(face.bb.top)),
                (int(face.bb.right), int(face.bb.bottom))
            ),
            outline='green',
            width=2
        )

        # text background
        draw.rectangle(
            (
                (int(face.bb.left - margin), int(face.bb.bottom) + margin),
                (int(face.bb.left + text_size[0] + margin), int(face.bb.bottom) + text_size[1] + 3 * margin)
            ),
            fill='black'
        )

        # text
        draw.text(
            (int(face.bb.left), int(face.bb.bottom) + 2 * margin),
            text,
            font=font
        )


In [15]:
img_path = 'images/sara_2/WIN_20211005_18_10_05_Pro.jpg'
img_path = 'images/Jeans/WIN_20211005_17_45_30_Pro.jpg'
img_path = '/home/daniel/Downloads/barack.jpeg' 
#img_path = '/home/daniel/Downloads/flaca.jpeg'
save_dir = 'results/'
preprocess = preprocessing.ExifOrientationNormalize()
img = Image.open(img_path)
filename = img.filename
img = preprocess(img)
img = img.convert('RGB')
faces = joblib.load(model_path)(img)
if not faces:
    print('No faces were found')
else:
    print('Faces found: ',str(len(faces)))
    draw_bb_on_img(faces, img)
    if save_dir is not None:
        basename = os.path.basename(filename)
        name = basename.split('.')[0]
        ext = basename.split('.')[1]
        img.save('{}_tagged.{}'.format(save_dir+name, ext))

Faces found:  1
0.480703141424811
